In [ ]:
# import libraries
from sklearn.feature_selection import SelectPercentile
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import feature_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn import neural_network
from sklearn import svm
from sklearn import kernel_ridge
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import RFECV
import csv
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
import xgboost as xgb

In [ ]:
# define functions for loading data and producing final CSV 

'''
loading training and test datasets
'''
def load_data():
    X_train = pd.read_csv("X_train.csv")
    X_test = pd.read_csv("X_test.csv")
    y_train = pd.read_csv("y_train.csv")
    
    X_train = X_train.drop('id', axis = 1)
    X_test = X_test.drop('id', axis = 1)
    y_train = y_train.drop('id', axis = 1)
    
    to_drop = to_be_eliminated(X_train)
    
    for i in range(len(to_drop)):
        X_train = X_train.drop(to_drop[i], axis = 1)
    
    for i in range(len(to_drop)):
        X_test = X_test.drop(to_drop[i], axis = 1)
    #print(len(X_test.columns))
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    return X_train, X_test, y_train


'''
eliminate highly correlated features
'''
def to_be_eliminated(df):
    # Create correlation matrix
    corr_matrix = df.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    return to_drop



'''
produce submission file
'''
def produce_solution(y):
    with open('out.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', lineterminator="\n")
        writer.writerow(['id', 'y'])
        for i in range(y.shape[0]):
            writer.writerow([float(i), y[i]])
              
'''
performance metric
BMAC = balanced_accuracy_score(y_true, y_pred)
'''


In [ ]:
X, X_test, y = load_data()
y = y.ravel()

kf = KFold(n_splits=10)
BMAC_scores = np.array([])

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
  
    #1. Zero Mean, Unit Variance
    print("Standardize data")
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # 2. Outlier detection
    print("Outlier Detection")
    lof = IsolationForest(n_neighbors=60, contamination=0.005)
    outliers = lof.fit_predict(X_train)

    unique, counts = np.unique(outliers, return_counts=True)
    count_dict = dict(zip(unique, counts))
    X_train = X_train[outliers == 1]
    y_train = y_train[outliers == 1]
    
    
    # 3. Feature selection 
    print("Feature Selection")
    select = SelectFromModel(RandomForestRegressor(n_estimators=200, random_state=42))
    select.fit(X_train, y_train)
    X_train = select.transform(X_train)
    
    print("Fitting the model")
    clf = xgb.XGBClassifier(random_state=42, learning_rate=0.1)
    clf.fit(X_train, y_train)
    
    #prediction 
    print("Predicting")
    X_test = select.transform(X_test)
    pred = reg.predict(X_test)
    
    #scoring
    
    score = r2_score(y_test, pred)
    #print(score)
    scores = np.append(scores,score)
    

